## Задача

- сделать анализ классификации текста с помощью LLM Qwen2-7B-Instruct
- сформировать classification_report с метриками
- замерить время получения предсказаний LLM и логрег
- сравнить метрики LLM и логрег
- описать результаты и сделать выводы

https://huggingface.co/Qwen/Qwen2-7B-Instruct - карточка модели<br>
Тут описано, что она из себя представляет и как её использовать.

In [1]:
# установить необходимые библиотеки
!pip install fuzzywuzzy
!pip install datasets
!pip install python-Levenshtein

In [2]:
from typing import Dict, Union, List
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [3]:
# функция для подбора промпта для llm
def prepare_message_for_llm(text: Union[str, List[str]], categories: List[str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if isinstance(text, str):
        text = [text]

    categories = ', '.join(categories)
    messages = []

    for msg in text:
        messages.append([{
            "role": "user",
            "content": f"Есть список категорий: {categories}. Нужно определить, \
            к какой категории относится следующий текст: {msg}. \
            Ответ напиши одним словом, только категория, как она указана в списке."

        }])

    return {'message_for_llm': messages}


In [4]:
# загрузка модели Qwen/Qwen2-7B-Instruct
llm = pipeline("text-generation", model="Qwen/Qwen2-7B-Instruct", return_full_text=False,
                max_new_tokens=512, device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
# загрузка датасета по выборкам train, validation, test
dataset = load_dataset("Davlan/sib200", "rus_Cyrl")

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 99
    })
    test: Dataset({
        features: ['index_id', 'category', 'text'],
        num_rows: 204
    })
})

In [7]:
dataset['train'] = dataset['train'].class_encode_column('category')
dataset['validation'] = dataset['validation'].class_encode_column('category')
dataset['test'] = dataset['test'].class_encode_column('category')

In [8]:
dataset['train']

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 701
})

In [9]:
dataset['train'][30]

{'index_id': 1441,
 'category': 3,
 'text': 'Британия заминировала международные воды, чтобы препятствовать судам входить в целые секторы океанов, подвергая опасности даже нейтральные суда.'}

In [10]:
categories = dataset['validation'].features['category'].names
categories

['entertainment',
 'geography',
 'health',
 'politics',
 'science/technology',
 'sports',
 'travel']

In [11]:
# добавить в список фичей колонку 'message_for_llm', которая получится в результате применения функции prepare_message_for_llm к текстам
train_messages = prepare_message_for_llm(dataset['train']['text'], categories)['message_for_llm']
validation_messages = prepare_message_for_llm(dataset['validation']['text'], categories)['message_for_llm']
test_messages = prepare_message_for_llm(dataset['test']['text'], categories)['message_for_llm']

dataset['train'] = dataset['train'].add_column(name="message_for_llm", \
                                               column=train_messages)
dataset['validation'] = dataset['validation'].add_column(name="message_for_llm", \
                                              column=validation_messages)
dataset['test'] = dataset['test'].add_column(name="message_for_llm", \
                                             column=test_messages)


In [12]:
dataset

DatasetDict({
    train: Dataset({
        features: ['index_id', 'category', 'text', 'message_for_llm'],
        num_rows: 701
    })
    validation: Dataset({
        features: ['index_id', 'category', 'text', 'message_for_llm'],
        num_rows: 99
    })
    test: Dataset({
        features: ['index_id', 'category', 'text', 'message_for_llm'],
        num_rows: 204
    })
})

In [13]:
print(dataset['train'][40])

{'index_id': 1941, 'category': 6, 'text': 'Широкие бульвары, здания со стеклянными фасадами современные торговые центры перемежаются местами  с традиционными крышами с красной черепицей, рынком XVIII века, старыми мечетями и церквями, хотя город и имеет больше атмосферу средиземноморской Европы, чем традиционной Турции.', 'message_for_llm': [{'content': 'Есть список категорий: entertainment, geography, health, politics, science/technology, sports, travel. Нужно определить,             к какой категории относится следующий текст: Широкие бульвары, здания со стеклянными фасадами современные торговые центры перемежаются местами  с традиционными крышами с красной черепицей, рынком XVIII века, старыми мечетями и церквями, хотя город и имеет больше атмосферу средиземноморской Европы, чем традиционной Турции..             Ответ напиши одним словом, только категория, как она указана в списке.', 'role': 'user'}]}


In [14]:
def predicted(name_dataset):
    pred = []

    for message in dataset[name_dataset]['message_for_llm']:
        generated_text = llm(message)[0]['generated_text']
        pred.append(generated_text)

    return pred

In [15]:
# получить предсказания для валидационной выборки и сформировать classification_report
def report(name_dataset, predicts):
    final_pred = []
    for predict in predicts:
        best_category = process.extractOne(predict, categories)[0]
        final_pred.append(categories.index(best_category))

    print(f'Classification_report для {name_dataset}')
    print(classification_report(dataset[name_dataset]['category'], final_pred))


In [ ]:
pred_val = predicted('validation')

In [17]:
report('validation', pred_val)

Classification_report для validation
              precision    recall  f1-score   support

           0       0.57      0.44      0.50         9
           1       0.71      0.62      0.67         8
           2       0.88      0.64      0.74        11
           3       0.79      0.79      0.79        14
           4       0.81      0.88      0.85        25
           5       0.73      0.92      0.81        12
           6       0.71      0.75      0.73        20

    accuracy                           0.76        99
   macro avg       0.74      0.72      0.73        99
weighted avg       0.76      0.76      0.75        99



In [ ]:
# сделать то же самое для тестовой выборки
pred_test = predicted('test')

In [ ]:
report('test', pred_test)

In [ ]:
# сделать вывод о полученных результатах
'''
Результаты модели выше, чем у регрессии. Модель более устойчива
к дисбалансу. Нужно учитывать, что важен сам запрос, так как неверно сформулированный промт
может привести к печальным результатам, и это будет вина не модели :)
'''